In [16]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


################################################################################
## Music Stuff
################################################################################
from musicUtils import *
from myMusicDBMap import myMusicDBMap


################################################################################
## Pandas Stuff
################################################################################
import sys
from collections import Counter
print("Python: {0}".format(sys.version))


################################################################################
## Prints
################################################################################
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-18 21:50:47.659889


# My Music DataBase

In [2]:
mdb = myMusicDBMap(debug=True)
#mdb.getFullDBData()()

Creating myMusicDBMap()
/Users/tgadfort/Documents/code/music
   Loading my music db map: /anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Counter({'Discogs': 3918, 'AllMusic': 3811, 'MusicBrainz': 3673, 'LastFM': 2777, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 169, 'CDandLP': 77, 'RateYourMusic': 29, 'MusicStack': 5, 'MetalStorm': 0})


In [23]:
from timeUtils import clock, elapsed
from myMusicDBMap import myMusicDBMap

class myMusicAnalysis:
    def __init__(self, mdb, debug=False):
        print("=================== Analysis of My Music ===================")
        self.debug = debug
        self.mdb   = mdb
        self.mdb.show()
        
        self.df = None
        self.results = {}

        
    def getArtistUnMatchedAlbums(self):
        retval         = {}


        ######################################################################
        #### Loop Over Prime Directories
        ######################################################################
        for primeDir in getPrimeDirectories():
            artistPrimeDirMap = getArtistPrimeDirMap(primeDir)
            print("UnMatched Albums: {0}".format(primeDir))
            #if primeDir.startswith("B"):
            #    break


            ######################################################################
            #### Loop Over Artist Name <-> Prime Map Items
            ######################################################################
            for artistName, artistPrimeDirs in artistPrimeDirMap.items():
                retval[artistName] = {"MyAlbums": [], "Todo": [], "Unknown": [], "Random": [], "Volumes": artistPrimeDirs}
                myAlbums           = {}
                for dirval in artistPrimeDirs:
                    myAlbums["MyAlbums"] = getMyMusicAlbums(dirval, returnNames=True)
                    myAlbums["Todo"]     = getMyTodoMusicAlbums(dirval)
                    myAlbums["Unknown"]  = getMyUnknownMusicAlbums(dirval)
                    myAlbums["Random"]   = getMyRandomMusic(dirval)

                    for key in myAlbums.keys():
                        retval[artistName][key] += myAlbums[key]

                for key in retval[artistName].keys():
                    retval[artistName][key] = len(retval[artistName][key])


        return retval



    def getArtistMatchedAlbums(self):
        albumTypes     = getAlbumTypes(keys=True)
        retval         = {}


        ######################################################################
        #### Loop Over Prime Directories
        ######################################################################
        for primeDir in getPrimeDirectories():
            artistPrimeDirMap = getArtistPrimeDirMap(primeDir) 
            print("Matched Albums: {0}".format(primeDir))
            #if primeDir.startswith("B"):
            #    break




            ######################################################################
            #### Loop Over Artist Name <-> Prime Map Items
            ######################################################################
            for artistName, artistPrimeDirs in artistPrimeDirMap.items():


                ######################################################################
                #### Get My Albums
                ######################################################################
                albumTypeResults = {}
                artistMatchDirs = [setDir(artistPrimeDir, 'Match', forceExist=False) for artistPrimeDir in artistPrimeDirs if isDir(artistPrimeDir, 'Match')]
                for albumType in albumTypes:
                    albumTypeResults[albumType] = []
                    for db in self.mdb.getDBs():
                        albumDBTypeNames = getAlbumTypes(db, albumType)
                        for albumDBTypeName in albumDBTypeNames:
                            for artistMatchDir in artistMatchDirs:
                                albumDBTypeDir = setDir(artistMatchDir, albumDBTypeName, forceExist=False)
                                if isDir(albumDBTypeDir):
                                    #print(albumDBTypeDir)
                                    albumTypeResults[albumType] += [getDirBasics(x)[-1] for x in findDirs(albumDBTypeDir)]
                                    #print(len(albumTypeResults),'\t',albumTypeResults)
                    albumTypeResults[albumType] = len(list(set(albumTypeResults[albumType])))
                retval[artistName] = albumTypeResults


        return retval
    
    
    def analyze(self):
        start, cmt = clock("Analyzing Artist Albums")

        artistUnMatchedAlbums = self.getArtistUnMatchedAlbums()
        artistMatchedAlbums   = self.getArtistMatchedAlbums()    

        self.results   = {}
        artistNames = set(list(artistUnMatchedAlbums.keys()) + list(artistMatchedAlbums.keys()))
        for artistName in sorted(artistNames):
            artistMatchedAlbumResults   = artistMatchedAlbums.get(artistName)
            artistUnMatchedAlbumResults = artistUnMatchedAlbums.get(artistName)
            #print(artistName,'\t',artistMatchedAlbumResults,'\t',artistUnMatchedAlbumResults)

            self.results[artistName]          = artistUnMatchedAlbumResults
            self.results[artistName]["Match"] = artistMatchedAlbumResults
            
        
        self.df = DataFrame(self.results).T            
            
        elapsed(start, cmt)
            
            
    def getDataFrame(self):
        if df is None:
            raise ValueError("DataFrame is not calculated!")
        return self.df
    
    
    def getResults(self):
        return self.results

In [12]:
mma = myMusicAnalysis(mdb)
mma.analyze()

=================== Analysis of My Music ===================
Counter({'Discogs': 3918, 'AllMusic': 3811, 'MusicBrainz': 3673, 'LastFM': 2777, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 169, 'CDandLP': 77, 'RateYourMusic': 29, 'MusicStack': 5, 'MetalStorm': 0})
Current Time is Sat Apr 18, 2020 21:38:57 for Analyzing Artist Albums
UnMatched Albums: A
UnMatched Albums: B
UnMatched Albums: C
UnMatched Albums: D
UnMatched Albums: E
UnMatched Albums: F
UnMatched Albums: G
UnMatched Albums: H
UnMatched Albums: I
UnMatched Albums: J
UnMatched Albums: K
UnMatched Albums: L
UnMatched Albums: M
UnMatched Albums: N
UnMatched Albums: Num
UnMatched Albums: O
UnMatched Albums: P
UnMatched Albums: Q
UnMatched Albums: R
UnMatched Albums: S
UnMatched Albums: T
UnMatched Albums: The
UnMatched Albums: U
UnMatched Albums: V
UnMatched Albums: W
UnMatched Albums: X
UnMatched Albums: Xtra
UnMatched Albums: Y
UnMatched Albums: Z
Matched Albums: A
Matched Albums: B
Matched Albums: C
Matched Albums: D
Mat

In [28]:
df = mma.getResults()

In [35]:
df["MyAlbums"][(df["MyAlbums"] > 0) & (df["MyAlbums"] < 5)]

*NSYNC                              2
2Gether                             1
2nd Avenue                          1
3 Doors Down                        1
311                                 2
38 Special                          1
3OH!3                               1
3X Krazy                            1
:wumpscut:                          2
A Flock Of Seagulls                 1
A Great Big World                   1
A-Mafia                             2
AC-DC                               1
AIR                                 1
AJR                                 1
ATB                                 1
Aaron Neville                       1
Accept                              3
Ace Of Base                         1
Acker Bilk                          1
Adryan Russ                         1
Aetherian                           4
Afrodite                            1
Al B. Sure!                         1
Al Bano & Romina Power              1
Al Lerner                           1
Alanis Moris

In [36]:
artist = "Rage Against The Machine"

In [37]:
mdb.getArtistData(artist)

{'Discogs': {'ID': '12212', 'Name': None},
 'AllMusic': {'ID': '0000863790', 'Name': None},
 'MusicBrainz': {'ID': '238939502661067675597894269866318097583',
  'Name': None},
 'AceBootlegs': None,
 'LastFM': {'ID': '11813736173', 'Name': None},
 'RateYourMusic': None,
 'DatPiff': {'ID': '2137739', 'Name': None},
 'RockCorner': {'ID': '142137739', 'Name': None},
 'CDandLP': None,
 'MusicStack': None,
 'MetalStorm': None}

In [24]:
import json
from collections import Counter

class showMyAlbums:
    def __init__(self, mma, debug=False):
        self.debug   = debug
        self.mma     = mma
        self.results = mma.getResults()

        
    def printAlbums(self, vals):
        retval = "".join(["{0: <4}".format(x) for x in vals])
        return retval


    def show(self):
        minMyAlbums   = 2
        minTodoAlbums = 5
        downloadCut   = 5

        ranks = {"Matches": Counter(), "Albums": Counter()}

        print("{0: <35}| {1: <10}| {2: <16}| {3: <10}| {4: <10}| {5: <10}| {6: <10}| {7: <12}| {8: <12}|".format("Artist", "Volumes", "Matches", "# Albums", "Todo", "Unknown", "Random", "No Match", "Many Albums"))
        print("{0: <35}| {1: <10}| {2: <16}| {3: <10}| {4: <10}| {5: <10}| {6: <10}| {7: <12}| {8: <12}|".format("------", "-------", "-------", "--------", "----", "-------", "------", "--------", "-----------"))

        for artistName, artistData in self.results.items():
            ### Artist Name
            print("{0: <35}".format(artistName), end="")

            ### Volumes
            volumeData = artistData["Volumes"]
            print("| {0: <10}".format(volumeData), end="")

            ### Matches
            dbData = artistData["Match"]
            print("| {0: <16}".format(self.printAlbums(dbData.values())), end="")

            ### Unmatched Albums
            myAlbumsData = artistData["MyAlbums"]
            print("| {0: <10}".format(myAlbumsData), end="")

            ### Todo Albums
            myTodoData = artistData["Todo"]
            print("| {0: <10}".format(myTodoData), end="")

            ### Unknown Albums
            myUnknownData = artistData["Unknown"]
            print("| {0: <10}".format(myUnknownData), end="")

            ### Random Albums
            myRandomData = artistData["Random"]
            print("| {0: <10}".format(myRandomData), end="")

            ### Check For No Match
            noMatchData = sum(dbData.values())
            noMatchText = "{0}..".format(artistName[:9])
            if noMatchData > 0:
                noMatchText = ""
            print("| {0: <12}".format(noMatchText), end="")

            ### Check For Many Albums
            manyAlbumsData = myAlbumsData
            manyAlbumsText = "{0}..".format(artistName[:9])
            if manyAlbumsData < 3:
                manyAlbumsText = ""
            print("| {0: <12}".format(manyAlbumsText), end="")

            ### Return
            print("|")

            ### Ranks
            ranks["Matches"][artistName] = sum(dbData.values())
            ranks["Albums"][artistName]  = myAlbumsData


            continue

            ### Check If We Need To Download More
            #maxDB = max([len(v) for k, v in artistData["DB"].items()])
            #if maxDB > 0:
            #    totalAlbums   = sum([v for k,v in artistData["Volumes"].items()])
            #    downloadRatio = int(round(100*(totalAlbums/maxDB),0))
            #else:
            downloadRatio = "---"
            print("| {0: <10}".format(downloadRatio), end="")

            download = ""
            if isinstance(downloadRatio, int):
                if downloadRatio < downloadCut and maxDB > 50:
                         download = artistName
            print("| {0: <20}".format(download), end="")



            ### Return
            print("")

In [27]:
sa = showMyAlbums(mma)
sa.show()

Artist                             | Volumes   | Matches         | # Albums  | Todo      | Unknown   | Random    | No Match    | Many Albums |
------                             | -------   | -------         | --------  | ----      | -------   | ------    | --------    | ----------- |
"Weird Al" Yankovic                | 1         | 13  0   0   0   | 0         | 1         | 0         | 0         |             |             |
'68 Comeback                       | 1         | 1   1   0   0   | 0         | 0         | 0         | 0         |             |             |
'68 Comeback Special               | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
*NSYNC                             | 1         | 1   1   0   0   | 2         | 2         | 0         | 2         |             |             |
-123 min.                          | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |

Andre Williams                     | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Andrea Marcovicci                  | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Andreas Johnson                    | 1         | 1   0   0   0   | 0         | 0         | 0         | 8         |             |             |
Andy Gibb                          | 2         | 0   0   0   0   | 2         | 0         | 0         | 0         | Andy Gibb.. |             |
Andy M. Stewart                    | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Angela Kaset                       | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Angelina Réaux                     | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |

Beanfield                          | 1         | 0   0   0   0   | 1         | 4         | 0         | 2         | Beanfield.. |             |
Beanie Sigel                       | 1         | 0   0   0   0   | 1         | 0         | 0         | 0         | Beanie Si.. |             |
Beastie Boys                       | 1         | 8   1   0   0   | 1         | 0         | 1         | 4         |             |             |
Beatrice Arthur                    | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Beaver Creek                       | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Beaver Nelson                      | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Bebe Rebozo                        | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |

Bob Ostertag                       | 1         | 6   0   0   1   | 0         | 0         | 0         | 0         |             |             |
Bob Rising                         | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Bob Seger                          | 1         | 20  4   1   0   | 2         | 0         | 0         | 7         |             |             |
Bobbie Gentry                      | 1         | 3   2   0   0   | 1         | 3         | 0         | 0         |             |             |
Bobby Birdman                      | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Bobby Charles                      | 1         | 0   0   0   0   | 1         | 0         | 0         | 0         | Bobby Cha.. |             |
Bobby Conn                         | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |

Carla Olson                        | 1         | 0   0   1   0   | 0         | 0         | 0         | 0         |             |             |
Carlos Santana                     | 1         | 4   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Carly Rae Jepsen                   | 1         | 1   0   0   0   | 1         | 0         | 0         | 1         |             |             |
Carly Simon                        | 1         | 19  0   0   0   | 0         | 4         | 0         | 0         |             |             |
Carnage                            | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Carnival In Coal                   | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Carole King                        | 2         | 9   1   0   0   | 0         | 1         | 0         | 0         |             |             |

...

The For Carnation                  | 1         | 1   0   1   0   | 0         | 0         | 0         | 0         |             |             |
The Forgotten                      | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Four Lovers                    | 1         | 1   0   1   0   | 0         | 0         | 0         | 0         |             |             |
The Four Seasons                   | 1         | 10  5   0   0   | 1         | 2         | 0         | 0         |             |             |
The Frantics                       | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Freedom Fighters               | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Freeze                         | 1         | 2   0   0   0   | 0         | 2         | 0         | 0         |             |             |

The Mono Men                       | 1         | 0   1   0   0   | 0         | 0         | 0         | 0         |             |             |
The Mooney Suzuki                  | 1         | 0   0   1   0   | 0         | 0         | 0         | 0         |             |             |
The Mopes                          | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Morlocks                       | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Morning Shakes                 | 1         | 1   0   1   0   | 0         | 0         | 0         | 0         |             |             |
The Mother Hips                    | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Moves                          | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |

The Volta Sound                    | 1         | 1   0   1   0   | 0         | 0         | 0         | 0         |             |             |
The Von Bondies                    | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Von Zippers                    | 1         | 0   0   1   0   | 0         | 0         | 0         | 0         |             |             |
The Wags                           | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Wailers                        | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Walkabouts                     | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |
The Walkmen                        | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |

Traluma                            | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Tram                               | 1         | 3   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Tramlaw                            | 1         | 1   0   1   0   | 0         | 0         | 0         | 0         |             |             |
Tramp                              | 1         | 0   1   0   0   | 0         | 0         | 0         | 0         |             |             |
Trampoline                         | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |
TransAtlantic                      | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Transcend                          | 1         | 4   0   0   0   | 0         | 0         | 0         | 0         |             |             |

Waltz For Debbie                   | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Wammo                              | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
War & Peace                        | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
WarHorse                           | 1         | 1   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Wardog                             | 1         | 2   0   0   0   | 0         | 0         | 0         | 0         |             |             |
Warlocks                           | 1         | 0   0   0   0   | 1         | 1         | 0         | 0         | Warlocks..  |             |
Warlord                            | 1         | 1   1   0   0   | 0         | 0         | 0         | 0         |             |             |

In [19]:

showAlbums(mma.getResults())

Artist                             | Volumes   | Matches         | # Albums  | Todo      | Unknown   | Random    | No Match    | Many Albums |
------                             | -------   | -------         | --------  | ----      | -------   | ------    | --------    | ----------- |
"Weird Al" Yankovic                | 1         Artist                             | Volumes   | Matches         | # Albums  | Todo      | Unknown   | Random    | No Match    | Many Albums |
------                             | -------   | -------         | --------  | ----      | -------   | ------    | --------    | ----------- |


AttributeError: 'dict_values' object has no attribute 'items'

In [ ]:
toget = []
for item in reversed(ranks["Matches"].most_common()):
    if item[1] == 0:
        toget.append(item[0])
toget
#saveFile(idata=toget, ifile="getRYM.p")

In [32]:
df = DataFrame(mma.getResults()).T            

In [33]:
df["Albums"]

KeyError: 'Albums'

In [ ]:
[(k,v) for k,v in ranks["Albums"].most_common() if k.startswith("S")]

In [ ]:
from random import random, randint

In [ ]:
central = [randint(5, 20) for x in range(100)]

In [ ]:
upper = [x+randint(0,7) for x in central]

In [ ]:
lower = [x-randint(0,7) for x in central]

In [ ]:
df = DataFrame([central, upper, lower]).T
df.columns = ["Central", "Upper", "Lower"]
from pandas import date_range
df.index = date_range("2020-01-01", periods=len(central))

In [ ]:
df.cumsum().plot()

df.to_csv("dummy.csv")

In [ ]:
date_range